In [1]:
# Import python packages for DL modeling and evaluation

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import time

from keras.preprocessing import sequence
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

from keras.models import Sequential
from keras.models import load_model

from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import BatchNormalization
from keras.layers import GlobalMaxPooling1D

from keras.utils import to_categorical

from sklearn import metrics
from sklearn import preprocessing

%matplotlib inline

Using TensorFlow backend.


In [2]:
# Load train/test dataset and build subject-trial column

train_dataset = pd.read_csv('../dataset/eeg_train_dataset.csv')
test_dataset = pd.read_csv('../dataset/eeg_test_dataset.csv')

train_dataset['Subject-Trial'] = train_dataset['Subject_id'].astype(str) + '-' + train_dataset['Trial_id'].astype(str)
test_dataset['Subject-Trial'] = test_dataset['Subject_id'].astype(str) + '-' + test_dataset['Trial_id'].astype(str)

In [3]:
# Check the dataset

train_dataset.head()

,Time-Series,FP1,FP2,F7,F8,AF1,AF2,FZ,F4,F3,...,P2,P1,CPZ,nd,Y,Alcohol,Exp,Subject_id,Trial_id,Subject-Trial
0,0,-8.921,0.834,-19.847,8.148,-2.146,1.129,-0.071,3.408,-0.092,...,-2.421,-4.313,-0.478,-8.901,-5.636,Y,S1,364,0,364-0
1,1,-8.433,3.276,-12.522,1.801,-2.146,0.641,-0.559,1.455,0.397,...,-3.886,-5.290,-0.966,-7.924,-2.706,Y,S1,364,0,364-0
2,2,-2.574,5.717,1.149,-2.594,-1.658,-0.336,-1.048,0.478,-1.068,...,-4.862,-5.290,-0.966,-3.042,1.689,Y,S1,364,0,364-0
3,3,5.239,7.670,14.821,-4.547,-0.682,-0.824,-0.559,0.966,-3.510,...,-3.886,-4.313,-0.966,4.771,5.595,Y,S1,364,0,364-0
4,4,11.587,9.623,20.681,-5.035,2.248,0.641,0.905,1.943,-5.463,...,-1.933,-2.360,-0.478,11.607,9.013,Y,S1,364,0,364-0


In [4]:
# We don't need to aggregate data when we use 1D-CNN model
# So, use matrix data directly

X_train = train_dataset.drop(['Time-Series', 'Alcohol', 'Exp', 'Subject_id', 'Trial_id', 'Subject-Trial'], axis=1)
y_train = train_dataset['Alcohol']

X_test = test_dataset.drop(['Time-Series', 'Alcohol', 'Exp', 'Subject_id', 'Trial_id', 'Subject-Trial'], axis=1)
y_test = test_dataset['Alcohol']

In [5]:
# Feature Scaling

X_train = preprocessing.scale(X_train)
X_test = preprocessing.scale(X_test)

In [6]:
# Change shape of train/test dataset for 1d-CNN
X_train_values = X_train.reshape(600, 256, 64).astype('float32')
X_test_values = X_test.reshape(600, 256, 64).astype('float32')

# Change target values to integer
y_train_list = []
y_test_list = []
y_to_zero = dict()
y_to_zero['Y'] = 0
y_to_zero['N'] = 1
for i in range(len(y_train)):
    if i % 256 == 0:
        y_train_list.append(y_to_zero[y_train[i]])
        y_test_list.append(y_to_zero[y_test[i]])

# Change target values to numpy array and categorical values
y_train_values = np.asarray(y_train_list).astype('float32')
y_test_values = np.asarray(y_test_list).astype('float32')

y_train_cat = to_categorical(y_train_values)
y_test_cat = to_categorical(y_test_values)

In [8]:
# Define simple and optimized 1D-CNN model
# For evaluation, basically repeat 10 times with 50 epochs

def general_1d_cnn(repeats = 10, epoch_num = 50):
    acc_list = []
    time_list = []
    for i in range(repeats):
        start_time = time.time()
        
        num_classes=2

        model = Sequential()
        model.add(Conv1D(filters=20, kernel_size=5, input_shape=(256, 64)))
        model.add(Dropout(0.2))
        model.add(Conv1D(filters=20, kernel_size=5))
        model.add(GlobalMaxPooling1D())
        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(num_classes, activation='softmax'))

        model.compile(loss='categorical_crossentropy',
                        optimizer='adam', metrics=['accuracy'])
        model.fit(X_train_values, y_train_cat, epochs=epoch_num)
        
        _, accuracy = model.evaluate(X_test_values, y_test_cat)
        acc_list.append(accuracy)
        time_list.append(time.time() - start_time)
    return acc_list, time_list

In [9]:
# Test 10 repeat with 30 epochs

acc_list, time_list = general_1d_cnn(repeats = 10, epoch_num = 30)

Epoch 1/30
600/600 [==============================] - 1s 1ms/step - loss: 1.0310 - acc: 0.5150
Epoch 2/30
600/600 [==============================] - 0s 418us/step - loss: 0.7803 - acc: 0.5633
Epoch 3/30
600/600 [==============================] - 0s 341us/step - loss: 0.6860 - acc: 0.6033
Epoch 4/30
600/600 [==============================] - 0s 380us/step - loss: 0.6193 - acc: 0.6633
Epoch 5/30
600/600 [==============================] - 0s 394us/step - loss: 0.5821 - acc: 0.6950
Epoch 6/30
600/600 [==============================] - 0s 375us/step - loss: 0.5153 - acc: 0.7517
Epoch 7/30
600/600 [==============================] - 0s 398us/step - loss: 0.4924 - acc: 0.7500
Epoch 8/30
600/600 [==============================] - 0s 418us/step - loss: 0.4660 - acc: 0.7817
Epoch 9/30
600/600 [==============================] - 0s 412us/step - loss: 0.4156 - acc: 0.8067
Epoch 10/30
600/600 [==============================] - 0s 380us/step - loss: 0.3752 - acc: 0.8617
Epoch 11/30
600/600 [==========

600/600 [==============================] - 0s 430us/step - loss: 0.0954 - acc: 0.9633
Epoch 24/30
600/600 [==============================] - 0s 405us/step - loss: 0.1444 - acc: 0.9517
Epoch 25/30
600/600 [==============================] - 0s 447us/step - loss: 0.1078 - acc: 0.9667
Epoch 26/30
600/600 [==============================] - 0s 441us/step - loss: 0.0919 - acc: 0.9683
Epoch 27/30
600/600 [==============================] - 0s 444us/step - loss: 0.1034 - acc: 0.9683
Epoch 28/30
600/600 [==============================] - 0s 341us/step - loss: 0.0799 - acc: 0.9683
Epoch 29/30
600/600 [==============================] - 0s 361us/step - loss: 0.1118 - acc: 0.9617
Epoch 30/30
600/600 [==============================] - 0s 348us/step
Epoch 1/30
600/600 [==============================] - 1s 2ms/step - loss: 0.9753 - acc: 0.5467
Epoch 2/30
600/600 [==============================] - 0s 400us/step - loss: 0.7546 - acc: 0.5450
Epoch 3/30
600/600 [==============================] - 0s 382us/st

600/600 [==============================] - 0s 415us/step - loss: 0.1804 - acc: 0.9400
Epoch 16/30
600/600 [==============================] - 0s 436us/step - loss: 0.1796 - acc: 0.9350
Epoch 17/30
600/600 [==============================] - 0s 428us/step - loss: 0.1617 - acc: 0.9400
Epoch 18/30
600/600 [==============================] - 0s 393us/step - loss: 0.1266 - acc: 0.9517
Epoch 19/30
600/600 [==============================] - 0s 421us/step - loss: 0.1550 - acc: 0.9600
Epoch 20/30
600/600 [==============================] - 0s 439us/step - loss: 0.1455 - acc: 0.9417
Epoch 21/30
600/600 [==============================] - 0s 439us/step - loss: 0.0921 - acc: 0.9717
Epoch 22/30
600/600 [==============================] - 0s 397us/step - loss: 0.0549 - acc: 0.9917
Epoch 23/30
600/600 [==============================] - 0s 390us/step - loss: 0.0547 - acc: 0.9917
Epoch 24/30
600/600 [==============================] - 0s 399us/step - loss: 0.0580 - acc: 0.9850
Epoch 25/30
600/600 [===========

600/600 [==============================] - 0s 457us/step - loss: 0.4481 - acc: 0.7967
Epoch 8/30
600/600 [==============================] - 0s 489us/step - loss: 0.4309 - acc: 0.8117
Epoch 9/30
600/600 [==============================] - 0s 456us/step - loss: 0.4029 - acc: 0.8067
Epoch 10/30
600/600 [==============================] - 0s 442us/step - loss: 0.3758 - acc: 0.8233
Epoch 11/30
600/600 [==============================] - 0s 383us/step - loss: 0.3333 - acc: 0.8533
Epoch 12/30
600/600 [==============================] - 0s 446us/step - loss: 0.2696 - acc: 0.8883
Epoch 13/30
600/600 [==============================] - 0s 454us/step - loss: 0.2644 - acc: 0.9000
Epoch 14/30
600/600 [==============================] - 0s 454us/step - loss: 0.2348 - acc: 0.9000
Epoch 15/30
600/600 [==============================] - 0s 450us/step - loss: 0.2186 - acc: 0.9167
Epoch 16/30
600/600 [==============================] - 0s 396us/step - loss: 0.1808 - acc: 0.9417
Epoch 17/30
600/600 [=============

In [10]:
# Check average accuracy with error

m, s = np.mean(acc_list), np.std(acc_list)
print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

Accuracy: 0.922% (+/-0.016)


In [13]:
# Check average time with error

m, s = np.mean(time_list), np.std(time_list)
print('Time: %.3fs (+/-%.3f)' % (m, s))

Time: 9.552s (+/-0.583)
